<a href="https://colab.research.google.com/github/daanicunha/projetoagentefinanceiro/blob/main/Projeto_IA_Alura_Planejamento_Financeiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
from IPython.display import HTML, Markdown

In [6]:
# Instalar Framework ADK de agentes do Google ################################################
%pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0

In [7]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [8]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [9]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [10]:
################################################
# --- Agente 1: Classificador de Despesas --- #
################################################
def agente_classificador(salario_str, despesas_fixa_str, despesas_variavel_str):
    classificador = Agent(
        name="agente_cassificador",
        model="gemini-2.0-flash",
        instruction=""" Voce é um agente responsável por coletar o salario, assim como as despesas fixas e variaveis do usuário.
        Voce precisa classificar as despesas recebidas pelo usuario.
        Sua funcao é garantir que todas as informacoes necessarias foram recebidas do usuario para que assim vocë possa seguir com a classificação das despesas.
        Utilize o google para pesquisar sobre a melhor forma de realizar a classificacao das despesas fixas e variaveis, e classifique-as corretamente.
        Tambem identifique qual a porcentagem de gasto de cada categoria, com base no salario enviado pelo usuario.
        """,
        description="Agente responsavel por classificar as despesas do usuario",
        tools=[google_search]
    )

    entrada_agente_classificador = f"Salario:{salario_str}\nDespesas Fixas: {despesas_fixa_str}\nDespesas Variaveis: {despesas_variavel_str}"

    # Executa o agente
    analise_despesas = call_agent(classificador, entrada_agente_classificador)
    return analise_despesas

# Exemplo de uso do agente coleta de dados:
# salario = [5000]
# despesas_fixas_exemplo = ["Aluguel: 1500", "Mensalidade Internet: 100", "Assinatura Streaming: 50"]
# despesas_variaveis_exemplo = ["Conta de Luz: 200", "Conta de Água: 80", "Supermercado: 500", "Transporte: 150"]
# resultado_analise = agente_coletor_dados(salario, despesas_fixas_exemplo, despesas_variaveis_exemplo)
# print("\n📊 Análise de Despesas:")
# print(resultado_analise)

In [22]:
###################################################
# --- Agente 2: Analista Financeiro Otimizador --- #
###################################################
def agente_analista_financeiro(analise_despesas, salario_str):
    analista_financeiro = Agent(
        name="agente_analista_financeiro",
        model="gemini-2.0-flash",
        instruction="""
           Você é um agente analista financeiro experiente, objetivo e estratégico.
           Sua missão é avaliar o orçamento mensal classificado pelo agente_classificador, que inclui o salário do usuário, a divisão das despesas fixas e variáveis e os percentuais de cada categoria em relação à renda.
           Com base nessa análise, compare o total de gastos com o salário e determine se o usuário está gastando acima, abaixo ou dentro do seu orçamento.
           Caso o usuário esteja gastando mais do que ganha, aponte as categorias com maior impacto financeiro e sugira formas práticas e personalizadas de reduzir esses gastos.
           Se o usuário estiver gastando menos do que ganha, proponha um plano simples e eficaz de investimento inicial, levando em conta perfis de risco moderado a conservador para iniciantes.
           Utilize pesquisas atualizadas no Google para embasar suas sugestões, tanto de economia quanto de investimento, priorizando fontes confiáveis como sites de educação financeira,
           corretoras reconhecidas e simuladores gratuitos.
            """,
        description="Agente que analisa a classificação do orçamento e sugere otimizações financeiras.",
        tools=[google_search]
    )

    entrada_agente_analista = f"""
    Salário: {salario_str}
    Classificacao e analise das despesas:
    {analise_despesas}
    """

    # Executa o agente
    sugestoes_otimizacao = call_agent(analista_financeiro, entrada_agente_analista)
    return sugestoes_otimizacao

# Exemplo de uso do agente analista financeiro:
# Assumindo que 'resultado_classificacao' é a saída do agente_classificador e 'salario_usuario' é o salário informado pelo usuário.
# resultado_classificacao_exemplo = """
# Despesas Fixas:
#   - Aluguel: 1500 (30% do salário)
#   - Mensalidade Internet: 100 (2% do salário)
#   - Assinatura Streaming: 50 (1% do salário)
# Total Despesas Fixas: 1650 (33% do salário)
#
# Despesas Variáveis:
#   - Conta de Luz: 200 (4% do salário)
#   - Conta de Água: 80 (1.6% do salário)
#   - Supermercado: 500 (10% do salário)
#   - Transporte: 150 (3% do salário)
# Total Despesas Variáveis: 930 (18.6% do salário)
#
# Total de Despesas: 2580 (51.6% do salário)
# Saldo Disponível: 2420 (48.4% do salário)
# """
# salario_usuario_exemplo = 5000
#
# sugestoes = agente_analista_financeiro(resultado_classificacao_exemplo, salario_usuario_exemplo)
# print("\n💰 Sugestões de Otimização Financeira:")
# print(sugestoes)

In [23]:
#################################################
# --- Agente 3: Agente de Lazer e Otimização --- #
#################################################

def agente_lazer_otimizacao(cep_usuario, sugestoes_otimizacao):
    agente_lazer = Agent(
        name="agente_lazer_otimizacao",
        model="gemini-2.0-flash",
        instruction=f"""
            Você é um Agente especializado em lazer e otimização de gastos, focado em encontrar opções de lazer próximas ao CEP {cep_usuario} do usuário, considerando {sugestoes_otimizacao}.
            Sua missão é pesquisar no Google Maps por lugares de lazer (como parques, cinemas, shoppings, restaurantes, bares, museus, etc.) na região do CEP fornecido.
            Após a pesquisa, identifique os cinco lugares mais bem avaliados.
            Para esses cinco lugares, tente encontrar informações sobre a média de preço de entrada, ingressos ou consumo. Se a média de preço não for facilmente acessível, mencione que essa informação não foi encontrada.
            Apresente os cinco lugares com maior avaliação, sua categoria (ex: parque, cinema), a média de preço (se encontrada) e a avaliação do Google Maps.
            Seu output deve ser claro, organizado por tópicos e direto ao ponto, utilizando pesquisas atualizadas no Google Maps e Google Search para obter as informações mais precisas.
            Considere o saldo disponível do usuário para sugerir opções que se encaixem no orçamento, mesmo que a média de preço não esteja disponível.
            """,
        description="Agente que pesquisa opções de lazer próximas ao CEP do usuário e apresenta a média de preço dos mais bem avaliados.",
        tools=[google_search]
    )

    entrada_agente_lazer = f"Pesquisar lugares de lazer próximos ao CEP {cep_usuario} com base no saldo disponível de {sugestoes_otimizacao}."

    # Executa o agente
    sugestoes_lazer = call_agent(agente_lazer, entrada_agente_lazer)
    return sugestoes_lazer

# Exemplo de uso do agente de lazer e otimização:
# Assumindo que 'cep_usuario_exemplo' é o CEP informado pelo usuário e 'saldo_disponivel_exemplo' é o saldo calculado.
# cep_usuario_exemplo = "01001-000"  # Exemplo de CEP
# saldo_disponivel_exemplo = 2420  # Exemplo de saldo disponível

# sugestoes = agente_lazer_otimizacao(cep_usuario_exemplo, saldo_disponivel_exemplo)
# print("\n🎉 Sugestões de Lazer:")
# print(sugestoes)

In [42]:
#################################################
# --- Agente 4: Agente de Planejamento Financeiro --- #
#################################################
def agente_planejamento_financeiro(sugestoes_otimizacao, sugestoes_lazer):
    agente_planejador = Agent(
        name="agente_planejamento_financeiro",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Agente de Planejamento Financeiro, sua tarefa é criar um plano financeiro mensal prático e eficaz para o usuário, baseado nas sugestões de otimização de orçamento
            e na classificação das despesas fornecidas pelo Agente Analista Financeiro.
            Seu plano deve ser estruturado da seguinte forma:
            1. **Resumo da Análise:** Apresente um resumo conciso da análise financeira e das sugestões de otimização recebidas pelo agente analista financeiro.
            2. **Estratégias de Monitoramento:** Ofereça sugestões diretas e práticas sobre como o usuário pode monitorar seus gastos ao longo do mês. Forneca um exemplo de como fazer isso, utilize dados fornecidos pelo analista financeiro.
            3. **Metas de Gastos Sugeridas:** Sugira metas de gastos por categoria, deixando claro que são apenas recomendações. Forneca um exemplo de como fazer isso, utilize dados fornecidos pelo analista financeiro.
            4. **Recursos Adicionais:** Indique fontes confiáveis (sites, videoaulas) para aprender mais sobre planejamento financeiro.
          O plano deve ser organizado, claro, objetivo e fácil de aplicar. Utilize pesquisas atualizadas para embasar suas sugestões e recursos.
            """,
        description="Agente que cria um plano financeiro mensal baseado nas sugestões de otimização.",
        tools=[google_search]
    )

    entrada_agente_planejador = f"""
    Sugestões de Otimização Financeira:
    {sugestoes_otimizacao}
    Sugestao de lazer:
    {sugestoes_lazer}

    """

    # Executa o agente
    plano_financeiro = call_agent(agente_planejador, entrada_agente_planejador)
    return plano_financeiro

# Exemplo de uso do agente de planejamento financeiro:
# Assumindo que 'sugestoes_otimizacao_exemplo' é a saída do agente_analista_financeiro e 'salario_usuario_exemplo' é o salário informado pelo usuário.
# sugestoes_otimizacao_exemplo = """
# Ótimas notícias! Sua análise financeira mostra um bom saldo disponível. Aqui estão algumas sugestões para otimizar ainda mais seus gastos:
# - Transporte: Considere alternativas para reduzir custos, como caronas ou transporte público em alguns dias.
# - Supermercado: Faça uma lista antes de ir ao supermercado para evitar compras por impulso e pesquise por promoções.
# """
# salario_usuario_exemplo = 5000

# plano = agente_planejamento_financeiro(sugestoes_otimizacao_exemplo, salario_usuario_exemplo)
# print("\n🗓️ Plano Financeiro para o Próximo Mês:")
# print(plano)

In [43]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("""💰 Olá, hoje vamos te ajudar no seu planejamento mensal para o próximo mes.\n
Queremos que vocë saiba gerenciar o seu dinheiro da melhor forma e aproveite para desfrutá-lo de forma consciente e responsavel❗💲\n""")

# --- Obter Informacoes Financeira do Usuário ---
# Solicitar cada informação separadamente
salario_str = input("▶️ Por favor, nos diga o seu salario mensal: \n")
despesas_fixa_str = input("▶️ Agora, liste suas despesas fixas, separadas por vírgula (Ex: Aluguel: 1500, Internet: 100): \n")
despesas_variavel_str = input("▶️ Liste suas despesas variaveis, separadas por vírgula (Ex: Luz: 200, Agua: 80, Supermercado: 500): \n")
cep_usuario = input("▶️ Por fim, nos diga o seu CEP: \n")

# Converter salário para um formato adequado (por exemplo, float ou integer)
# Você pode adicionar tratamento de erro aqui para garantir que a entrada seja um número
salario = float(salario_str) # Converter para float

# Dividir as strings de despesas em listas
# Remover espaços em branco de cada item
despesas_fixa = [item.strip() for item in despesas_fixa_str.split(',')]
despesas_variavel = [item.strip() for item in despesas_variavel_str.split(',')]


# Inserir lógica do sistema de agentes ################################################
# A variável 'topico' não está definida no código fornecido.
# Parece que sobrou de outro snippet de código.
# Devemos chamar os agentes com os dados financeiros coletados.

# Chamar os agentes sequencialmente, passando a saída do agente anterior para o próximo
analise_despesas = agente_classificador(salario_str, despesas_fixa_str, despesas_variavel_str)
sugestoes_otimizacao = agente_analista_financeiro(analise_despesas, salario_str)
# Adicionar a chamada para o agente_lazer_otimizacao to generate sugestoes_lazer
sugestoes_lazer = agente_lazer_otimizacao(cep_usuario, sugestoes_otimizacao)
plano_financeiro = agente_planejamento_financeiro(sugestoes_otimizacao, sugestoes_lazer)

# Exibir o plano financeiro final
print("\n🗓️ Plano Financeiro para o Próximo Mês:")

# Organizar resposta
from IPython.display import Markdown

# Retorno para o usuario
display(to_markdown(plano_financeiro + "\n\n🎉 Sugestões de Lazer:\n" + sugestoes_lazer))

💰 Olá, hoje vamos te ajudar no seu planejamento mensal para o próximo mes.
 
Queremos que vocë saiba gerenciar o seu dinheiro da melhor forma e aproveite para desfrutá-lo de forma consciente e responsavel❗💲

▶️ Por favor, nos diga o seu salario mensal: 
5000
▶️ Agora, liste suas despesas fixas, separadas por vírgula (Ex: Aluguel: 1500, Internet: 100): 
Aluguel: 1500, Internet: 100
▶️ Liste suas despesas variaveis, separadas por vírgula (Ex: Luz: 200, Agua: 80, Supermercado: 500): 
Luz: 200, Agua: 80, Supermercado: 500
▶️ Por fim, nos diga o seu CEP: 
04733200

🗓️ Plano Financeiro para o Próximo Mês:


> Com prazer! Aqui está um plano financeiro mensal prático e eficaz, baseado nas suas sugestões de otimização de orçamento:
> 
> **1. Resumo da Análise:**
> 
> Com base nas informações fornecidas, você possui uma situação financeira confortável, com uma sobra mensal de R$2620, representando 52.4% do seu salário de R$5000. Isso indica uma excelente oportunidade para começar a investir e garantir um futuro financeiro mais seguro. As sugestões de investimento inicial recomendam opções conservadoras a moderadas, como Tesouro Direto, Fundos de Renda Fixa, CDBs, LCIs e LCAs, visando a segurança e a diversificação dos seus investimentos.
> 
> **2. Estratégias de Monitoramento:**
> 
> Para garantir que você continue no caminho certo, é fundamental monitorar seus gastos ao longo do mês. Uma forma simples e eficaz de fazer isso é utilizando uma planilha ou aplicativo de controle financeiro.
> 
> *   **Exemplo prático:**
> 
>     *   **Receita Mensal:** R$5000
>     *   **Gastos:**
>         *   Moradia: R$0 (já pago)
>         *   Alimentação: R$500
>         *   Transporte: R$200
>         *   Lazer: R$300
>         *   Outros: R$380
>         *   **Total de Gastos:** R$1380
>     *   **Investimentos:** R$1000
>     *   **Sobra:** R$2620
> 
> *   **Como monitorar:**
> 
>     1.  **Registre todos os gastos:** Anote cada despesa, por menor que seja.
>     2.  **Categorize os gastos:** Divida os gastos em categorias (alimentação, transporte, lazer, etc.).
>     3.  **Compare com o orçamento:** Verifique se os gastos estão dentro das metas estabelecidas.
>     4.  **Analise os resultados:** Identifique áreas onde é possível economizar.
>     5.  **Ajuste o orçamento:** Faça ajustes no orçamento conforme necessário.
> 
> **3. Metas de Gastos Sugeridas:**
> 
> As metas de gastos sugeridas são apenas recomendações, e você pode ajustá-las de acordo com suas prioridades e necessidades.
> 
> *   **Exemplo:**
> 
>     *   **Alimentação:** Meta de R$700 por mês.
>     *   **Transporte:** Meta de R$300 por mês.
>     *   **Lazer:** Meta de R$500 por mês (incluindo as opções de lazer perto de seu CEP).
>     *   **Outros:** Meta de R$380 por mês.
> 
> *   **Como definir suas metas:**
> 
>     1.  **Analise seus gastos:** Veja onde você está gastando mais dinheiro.
>     2.  **Defina prioridades:** Decida quais são as áreas mais importantes para você.
>     3.  **Estabeleça limites:** Defina limites de gastos para cada categoria.
>     4.  **Seja realista:** Defina metas que você possa alcançar.
> 
> **4. Recursos Adicionais:**
> 
> Para aprender mais sobre planejamento financeiro e investimentos, você pode consultar as seguintes fontes confiáveis:
> 
> *   **Sites:**
>     *   Tesouro Direto ([https://www.tesourodireto.com.br/](https://www.tesourodireto.com.br/)): Informações sobre títulos públicos.
>     *   Calculadora do Cidadão do Banco Central ([https://www.bcb.gov.br/](https://www.bcb.gov.br/)): Ferramentas para calcular a rentabilidade de investimentos.
> *   **Vídeos:**
>     *   Vídeos do canal Me Poupe! no YouTube: Dicas práticas sobre finanças pessoais e investimentos.
>     *   Vídeos do canal Nath Finanças no YouTube: Conteúdo educativo sobre finanças para iniciantes.
> *   **Livros:**
>     *   "O Pai Rico, Pai Pobre" de Robert Kiyosaki: Abordagem sobre mentalidade financeira e investimentos.
>     *   "Dinheiro não cai do céu" de Álvaro Modernell: Guia prático de educação financeira para jovens.
> 
> Lembre-se que este é apenas um plano inicial, e você pode adaptá-lo de acordo com suas necessidades e objetivos financeiros. Ao seguir estas dicas e dedicar um tempo para aprender mais sobre finanças pessoais, você estará no caminho certo para construir um futuro financeiro mais seguro e próspero.
> 
> 
> 
> 🎉 Sugestões de Lazer:
> Com certeza! Com uma sobra mensal de R$2620, você tem flexibilidade para escolher opções de lazer que se encaixem no seu orçamento e ainda investir. Veja algumas sugestões de lugares bem avaliados próximos ao CEP 04733-200, com foco em custo-benefício:
> 
> 
> Com base nas pesquisas, aqui estão cinco opções de lazer bem avaliadas próximas ao CEP 04733-200, considerando diferentes categorias e faixas de preço:
> 
> 1.  **Shopping Mais Shopping:** Opção de compras e lazer com diversas lojas e restaurantes. Avaliação geralmente positiva, mas a média de preços pode variar dependendo das lojas e restaurantes escolhidos.
> 2.  **Boa Vista Shopping:** Similar ao Shopping Mais Shopping, oferece diversas opções de lojas, alimentação e entretenimento. A média de preços também varia conforme as escolhas de consumo.
> 3.  **Parque Severo Gomes:** Uma área verde para relaxar, caminhar e aproveitar a natureza. Geralmente gratuito, sendo uma opção econômica e bem avaliada para atividades ao ar livre.
> 4.  **Lanches Los Bar:** Localizado na Avenida Adolfo Pinheiro, oferece lanches e petiscos. Não foi possível encontrar a média de preços, mas bares e lanchonetes costumam ter opções acessíveis.
> 5.  **Opções de Restaurantes:** Explore restaurantes na Av. Adolfo Pinheiro, como o Terraço Jardins - Hotel Renaissance e Jardim da Vila. Os preços variam de €€€€ (mais caros) a opções mais acessíveis.
> 
> Considerando seu orçamento, sugiro explorar o Parque Severo Gomes para uma opção gratuita e relaxante, ou experimentar os lanches e restaurantes na Av. Adolfo Pinheiro, que oferecem diversas faixas de preço. Os shoppings também são boas alternativas, mas é importante estar atento aos gastos com lojas e alimentação.
> 
